In [20]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-20
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-22
@Title : To print twitter records as dictionaries

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-11-20\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-11-20\n@Title : To print twitter records as dictionaries\n\n'

In [21]:
import boto3
import json
import os
import pandas as pd

In [22]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')
KINESIS_DATA_STREAM = 'kds-twitter-sda'
TABLE_NAME = 'WorldCup2022Tweets'

### Fucntions to connect Kinesis Stream , get shards and get tweets from stream as pandas dataframe.

In [23]:
# df = pd.DataFrame()
list_of_records = []

def look_end_of_stream(kinesis_client,my_shard_id):
    """

    Description:
        A shard iterator specifies the shard position from which to start reading data records sequentially. 
        The position is specified using the sequence number of a data record in a shard. A sequence number is the identifier associated with every record ingested in the stream, 
        and is assigned when a record is put into the stream. 
        We use ShardIteratorType as LATEST to look
        at the end of the stream for new incoming data.
    Parameter:
        Kinesis kinesis_client,
        string shard_id
    Return:
        string my_shard_iterator

    """

    
    shard_iterator = kinesis_client.get_shard_iterator(StreamName=KINESIS_DATA_STREAM,
                                                    ShardId=my_shard_id,
                                                    ShardIteratorType='LATEST')
    my_shard_iterator = shard_iterator['ShardIterator']
    return my_shard_iterator                                                  

def connect_to_stream(kinesis_client):
    """

    Description:
        To Describe the specified Kinesis data stream. 
        The information returned includes the stream name, Amazon Resource Name (ARN), creation time, enhanced metric configuration, and shard map.
    Parameter:
        Kinesis kinesis_client
    Return:
        string shard_id

    """

    response = kinesis_client.describe_stream(StreamName=KINESIS_DATA_STREAM)
    my_shard_id = response['StreamDescription']['Shards'][0]['ShardId']

    return my_shard_id

def get_twitter_records(kinesis_client,my_shard_iterator):
    """

    Description:
       Gets data records from a Kinesis data stream's shard.
        By speciying shard iterator using the ShardIterator parameter, 
        the shard iterator specifies the position in the shard from which you want to start reading data records sequentially.

        After getting records from the shard we convert it into a list of dictionaries
    Parameter:
        Kinesis kinesis_client,
        string shard_iterator
    Return:
        list records

    """
    # count = 0
    record_response = kinesis_client.get_records(
        ShardIterator=my_shard_iterator,
        Limit=100
    )
    # list_of_records = []
    while 'NextShardIterator' in record_response:
        # read up to 100 records at a time from the shard number
        record_response = kinesis_client.get_records(
        ShardIterator=record_response['NextShardIterator'],
        Limit=100
        )
        # Print only if we have something
        if(record_response['Records']):
            print (record_response)

            list_of_records.append(record_response['Records'][0])

def records_to_dict(list_of_records):
    """

    Description:
        Extracting values of nested dictionary from 'Data' key from list_of_records
    Parameter:
        list_of_records
    Return:
        list records

    """

    records = []

    for d in list_of_records:
        new_rec = {}
        for k in d:
            if k == 'Data':
                data_dict = json.loads(d[k].decode('utf-8'))
                for tweet_info in data_dict:
                    new_rec[tweet_info] = data_dict[tweet_info]
            else:    
                new_rec[k] = d[k]

        records.append(new_rec)

    return records    

def tweets_as_df(records):
    """

    Description:
        Convert list of dictionaries 'records' into a pandas dataframe
    Parameter:
        list records
    Return:
        Dataframe df

    """
    df = pd.DataFrame()
    for r in records:
        # print(record)
        df = df.append(r, ignore_index=True)

    return df    

def word_occurences(text):
    """

    Description:
        to count occurence of each word in a string and return result as a dictionary.
    Parameter:
        string text
    Return:
        dictionary counts

    """
    counts = dict()
    words = text.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

def count_words_of_tweet(df):
    
    """

    Description:
        To add two columns : 'word_count_on_text' which will have count of number of words in a tweet 
        and 'word_occurences_of_text' which will have dictionary of word occurences in the tweet.
    Parameter:
        Dataframe df
    Return:
        Dataframe df

    """
    list_of_word_count = []
    for index,row in df.iterrows():
        list_of_word_count.append(len(row['tweet_text'].strip().split()))

    df['word_count_on_text'] = list_of_word_count

    # list_tweet_text = list(df['tweet_text'])
    word_occurences_on_text = []
    for index,row in df.iterrows():
        word_occurences_on_text.append(word_occurences(row['tweet_text']))
    # for i in range(len(list_tweet_text)):
    #     word_occurences_on_text.append(word_occurences(list_tweet_text[i]))

    df['word_occurences_on_text'] = word_occurences_on_text

    return df



### Functions to add pandas dataframe to DynamoDB Table

In [24]:
def connect_to_dynamoDB():
    """

    Description:
        To connect to AWS DynamoDB service.
    Parameter:
        No parameters
    Return:
        ServiceResource dyDB

    """
    dyDB =  boto3.resource('dynamodb')
    return dyDB

def create_items(table,twitter_records):
    """

    Description:
        To create items in a table of DynamoDB.
    Parameter:
        dynamodb.table table
    Return:
        No values returned.

    """
    table.put_item(
        Item={
                'tweet_id': twitter_records[0],
                'ArrivalTimestamp': twitter_records[1],
                'tweet_text': twitter_records[2],
                'user_id': twitter_records[3],
                'display_name': twitter_records[4],
                'user_name': twitter_records[5],
                'word_count_on_text': twitter_records[6],
                'word_occurences_on_text': twitter_records[6]
            }
    )

def create_dynamoDB_Table(dyDB):
    """

    Description:
        To create a dynamoDB table if it doen't exist
    Parameter:
        ServiceResource dyDB
    Return:
        no value returned

    """
    
    dynamodb_client = boto3.client('dynamodb')
    table_name = TABLE_NAME
    existing_tables = dynamodb_client.list_tables()['TableNames']

    if table_name not in existing_tables:
        # Create the DynamoDB table.
        dyDB.create_table(
            TableName=TABLE_NAME,
            KeySchema=[
                {
                    'AttributeName': 'tweet_id',
                    'KeyType': 'HASH'
                },
                {
                    'AttributeName': 'ArrivalTimestamp',
                    'KeyType': 'RANGE'
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'tweet_id',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'ArrivalTimestamp',
                    'AttributeType': 'S'
                },
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )

### Caling Kinesis Custom Functions

In [25]:
session = boto3.Session()
kinesis_client = session.client('kinesis')

my_shard_id = connect_to_stream(kinesis_client)

my_shard_iterator = look_end_of_stream(kinesis_client,my_shard_id)

list_of_records = get_twitter_records(kinesis_client,my_shard_iterator)
# print(list_of_records)


{'Records': [{'SequenceNumber': '49635394015087978218110013620426680671690839458855256066', 'ApproximateArrivalTimestamp': datetime.datetime(2022, 11, 22, 12, 40, 57, 774000, tzinfo=tzlocal()), 'Data': b'{"tweet_id": "1594951526619176961", "tweet_text": "RT @AmnestyIran: As the world\'s attention is turned to the #ENGIRN game, let\'s remember the faces of children killed by Iran\'s security for\\u2026", "user_id": "965136351455715328", "user_name": "resa_ds", "user_username": "leyla_677"}', 'PartitionKey': 'dd3b40e6-27ce-4881-abe3-ad625c2e484e'}, {'SequenceNumber': '49635394015087978218110013620556035734389604780548816898', 'ApproximateArrivalTimestamp': datetime.datetime(2022, 11, 22, 12, 40, 57, 982000, tzinfo=tzlocal()), 'Data': b'{"tweet_id": "1594951524303753216", "tweet_text": "Mexico vs Poland Live Stream Free, TV channel, start time, Team News and streams\\n\\n\\ud83d\\udd34\\ud835\\udc0b\\ud835\\udc22\\ud835\\udc2f\\ud835\\udc1e \\ud835\\udc0b\\ud835\\udc22\\ud835\\udc27\\ud83

KeyboardInterrupt: 

### Calling DynamoDB Custom Functions

In [26]:
records = records_to_dict(list_of_records)

df = tweets_as_df(records)

df = count_words_of_tweet(df)

dyDB = connect_to_dynamoDB()

create_dynamoDB_Table(dyDB)
table = dyDB.Table(TABLE_NAME)
table.wait_until_exists()

# Print out some data about the table.
print(f"DateTime creation of Table : {table.creation_date_time}")
print(f"No. of items in the table :{table.item_count}")

# table = dyDB.Table('EmployeeBOTO3')

for index,row in df.iterrows():
    record_as_tuple = row['tweet_id'],str(row['ApproximateArrivalTimestamp']),row['tweet_text'],row['user_id'],row['user_name'],row['user_username'],row['word_count_on_text'],dict(row['word_occurences_on_text'])
    record_as_list = list(record_as_tuple)
    create_items(table,record_as_list)


C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_1412\355355768.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_1412\355355768.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_1412\355355768.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_1412\355355768.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\

DateTime creation of Table : 2022-11-22 12:55:30.337000+05:30
No. of items in the table :0


In [27]:
df.head(10)

,SequenceNumber,ApproximateArrivalTimestamp,tweet_id,tweet_text,user_id,user_name,user_username,PartitionKey,word_count_on_text,word_occurences_on_text
0,4963539401508797821811001362042668067169083945...,2022-11-22 12:40:57.774000+05:30,1594951526619176961,RT @AmnestyIran: As the world's attention is t...,965136351455715328,resa_ds,leyla_677,dd3b40e6-27ce-4881-abe3-ad625c2e484e,23,"{'RT': 1, '@AmnestyIran:': 1, 'As': 1, 'the': ..."
1,4963539401508797821811001362078573164011638432...,2022-11-22 12:40:58.186000+05:30,1594951524769300480,Isalamic Republic and IRGC have sent armed tro...,1462890041437474817,هەموو ژیناین,zhinakan,dd3b40e6-27ce-4881-abe3-ad625c2e484e,36,"{'Isalamic': 1, 'Republic': 1, 'and': 3, 'IRGC..."
2,4963539401508797821811001362091629562863476427...,2022-11-22 12:40:58.397000+05:30,1594951523317981184,"I’m sorry, but #Latto’s version of #BlickBlick...",3451959143,O.G.,whatisgooding,dd3b40e6-27ce-4881-abe3-ad625c2e484e,33,"{'I’m': 1, 'sorry,': 1, 'but': 1, '#Latto’s': ..."
3,4963539401508797821811001362104323283969430033...,2022-11-22 12:40:58.603000+05:30,1594951525843013633,NETHERLANDS 🇳🇱 \n\nSHOP EASY BALL HARD 🟢🟢🟢\n\n...,1345500487374811137,SPORTSCONTINENT,SPORTSCONTINEN2,dd3b40e6-27ce-4881-abe3-ad625c2e484e,15,"{'NETHERLANDS': 1, '🇳🇱': 1, 'SHOP': 1, 'EASY':..."
4,4963539401508797821811001362118346823476959732...,2022-11-22 12:40:58.814000+05:30,1594951529488060416,RT @binary_x: ❓Which team will lift the trophy...,1483936314110406656,Newbiland,newbiland,dd3b40e6-27ce-4881-abe3-ad625c2e484e,20,"{'RT': 1, '@binary_x:': 1, '❓Which': 1, 'team'..."
5,4963539401508797821811001362133941966549988455...,2022-11-22 12:40:59.071000+05:30,1594951534437187586,@TopGoal_NFT Argentina win,1369619773106098176,Shahbaz Ali,Shahbaz033432,dd3b40e6-27ce-4881-abe3-ad625c2e484e,3,"{'@TopGoal_NFT': 1, 'Argentina': 1, 'win': 1}"
6,4963539401508797821811001362147602828311633765...,2022-11-22 12:40:59.300000+05:30,1594951532398940160,Started as a “10” in essence but most of the p...,358769498,Rey Quraishi,rey_qur,dd3b40e6-27ce-4881-abe3-ad625c2e484e,30,"{'Started': 1, 'as': 1, 'a': 1, '“10”': 1, 'in..."
7,4963539401508797821811001362292432141501466353...,2022-11-22 12:41:01.112000+05:30,1594951544495304704,RT @RoadQatar_WC: 🔥🔥🔥$ROAD SUCCESSFUL LISTING ...,1593402442393489408,Billa,Billa032,dd3b40e6-27ce-4881-abe3-ad625c2e484e,20,"{'RT': 1, '@RoadQatar_WC:': 1, '🔥🔥🔥$ROAD': 1, ..."
8,4963539401508797821811001362361703590965384605...,2022-11-22 12:41:02.057000+05:30,1594951545849888768,• THREAD\n\nArgentina Portraits - FIFA World C...,1200810243372015617,HQ VITA,HQVita,dd3b40e6-27ce-4881-abe3-ad625c2e484e,17,"{'•': 1, 'THREAD': 1, 'Argentina': 1, 'Portrai..."
9,4963539401508797821811001362380441941169411364...,2022-11-22 12:41:02.267000+05:30,1594951545069658117,Weldon @qatar\nYou Slapped the Illuminati &amp...,1505203001702686721,Zain Bajwa,Zain_BajwaPTI,dd3b40e6-27ce-4881-abe3-ad625c2e484e,26,"{'Weldon': 1, '@qatar': 1, 'You': 1, 'Slapped'..."
